In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import folium
from folium import plugins
from folium.plugins import HeatMap
from scipy.interpolate import interp1d
from scipy.interpolate import make_interp_spline, BSpline

In [9]:
df_global=pd.read_csv('Datasets/GlobalTemperatures.csv')
df_byCountry= pd.read_csv('Datasets/GlobalLandTemperaturesByCity.csv')
df_byState= pd.read_csv('Datasets/GlobalLandTemperaturesByState.csv')
df_major=pd.read_csv('Datasets/Filtered_Major_Cities.csv')
df_city=pd.read_csv('Datasets/GlobalLandTemperaturesByCity.csv')
df_emission =pd.read_csv('Datasets/emission data.csv')

In [10]:
df_global['dt'] = pd.to_datetime(df_global.dt)
df_landAverage = df_global.iloc[:, :3] 
df_landAverage = df_landAverage.reset_index(drop=True)
df_global = df_global.dropna(axis=0)
df_global = df_global.reset_index(drop=True)

In [11]:
df_byCountry['dt'] = pd.to_datetime(df_byCountry.dt)
non_country_lst=['Antarctica', 'Africa', 'Asia', 'Europe', 'North America', 'South America','Denmark', 
                 'France', 'Netherlands','United Kingdom','Åland', 'American Samoa', 'Anguilla', 'Baker Island', 
                 'Bonaire', 'Saint Eustatius And Saba', 'British Virgin Islands', 'Cayman Islands', 
                 'Christmas Island', 'Falkland Islands (Islas Malvinas)', 'Faroe Islands', 'French Guiana', 
                 'French Southern And Antarctic Lands', 'Gaza Strip', 'Greenland', 'Guadeloupe', 'Guam', 'Guernsey', 
                 'Heard Island And Mcdonald Islands', 'Isle Of Man', 'Jersey', 'Kingman Reef', 'Macau', 'Martinique', 
                 'Mayotte', 'Montserrat', 'New Caledonia', 'Northern Mariana Islands', 'Palmyra Atoll', 'Reunion', 
                 'Saint Martin', 'Saint Pierre And Miquelon', 'South Georgia And The South Sandwich Islands', 
                 'Turks and Caicas Islands', 'Virgin Islands', 'Western Sahara']
continents = ['Antarctica', 'Africa', 'Asia', 'Australia', 'Europe', 'North America', 'South America']
df_continents = df_byCountry[df_byCountry.Country.isin(continents)]
df_continents.reset_index(drop=True)


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1841-01-01,21.432,3.286,Adelaide,Australia,34.56S,138.16E
1,1841-02-01,22.087,2.458,Adelaide,Australia,34.56S,138.16E
2,1841-03-01,18.859,3.547,Adelaide,Australia,34.56S,138.16E
3,1841-04-01,15.033,1.884,Adelaide,Australia,34.56S,138.16E
4,1841-05-01,12.864,1.481,Adelaide,Australia,34.56S,138.16E
...,...,...,...,...,...,...,...
28473,2013-05-01,16.947,0.154,Wollongong,Australia,34.56S,151.78E
28474,2013-06-01,15.911,0.176,Wollongong,Australia,34.56S,151.78E
28475,2013-07-01,15.518,0.178,Wollongong,Australia,34.56S,151.78E
28476,2013-08-01,16.126,0.325,Wollongong,Australia,34.56S,151.78E


In [12]:
df_countries = df_byCountry[~df_byCountry.Country.isin(non_country_lst)]
val_d = df_countries.loc[df_countries['Country'] == 'Denmark (Europe)', 'Country'] 
val_d= 'Denmark'
val_f = df_countries.loc[df_countries['Country'] == 'France (Europe)', 'Country'] 
val_f = 'France'
val_n = df_countries.loc[df_countries['Country'] == 'Netherlands (Europe)', 'Country'] 
val_n='Netherlands'
val_uk=df_countries.loc[df_countries['Country'] == 'United Kingdom (Europe)', 'Country'] 
val_uk='United Kingdom'


df_countries = df_countries.reset_index(drop=True)

df_countries['day'] = df_countries['dt'].dt.day
df_countries['month'] = df_countries['dt'].dt.month
df_countries['year'] = df_countries['dt'].dt.year

country_oldestDate = df_countries.groupby(['Country']).min()
max(country_oldestDate['year'].values)

1882

In [13]:
df_country1894 = df_countries.copy()
df_country1894 = df_country1894[(df_countries['year']>=1894)&(df_countries['year']<2013)]
df_country1894.reset_index(drop=True)
country_temp = df_country1894['Country'].unique()

In [14]:
countries = np.unique((df_country1894['Country']))
mean_temp=[]

for country in countries:
    mean_temp.append(df_country1894[(df_country1894['Country']==country)&(df_country1894['year']==2012)]['AverageTemperature'].mean())
    
data = [dict(
        type = 'choropleth',
        locations = countries,
        z = mean_temp,
        locationmode = 'country names',
        text = countries,
        colorscale = [[0.0, "rgb(49,54,149)"],
                [0.1111111111111111, "rgb(69,117,180)"],
                [0.2222222222222222, "rgb(116,173,209)"],
                [0.3333333333333333, "rgb(171,217,233)"],
                [0.4444444444444444, "rgb(224,243,248)"],
                [0.5555555555555556, "rgb(254,224,144)"],
                [0.6666666666666666, "rgb(253,174,97)"],
                [0.7777777777777778, "rgb(244,109,67)"],
                [0.8888888888888888, "rgb(215,48,39)"],
                [1.0, "rgb(165,0,38)"]],
        marker = dict(
            line = dict(color = 'rgb(0,0,0)', width = 1)),
            colorbar = dict(autotick = True, tickprefix = '', 
            title = 'Average\nTemperature,\n°C'),
            font=dict(family='Times New Roman', size=18, color='black')
        size=40;
        )]

SyntaxError: invalid syntax (Temp/ipykernel_6268/685979860.py, line 28)

In [ ]:
layout = dict(
    title = 'Average Land Temperature by Country in 2012',
    font=dict(family='Times New Roman', size=20, color='black'),
    geo = dict(
        showframe = True,
        showocean = True,
        oceancolor = 'rgb(0,0,0)',
        projection = dict(
                type = 'orthographic',
            rotation = dict(
                lon = 60,
                lat = 10),
            ),
            lonaxis =  dict(
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)'
            ),
            lataxis = dict(
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)'
            )
        ),
    )

In [ ]:
fig=dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='worldsphere')